# Lecture 20 – Data 100, Fall 2024

Data 100, Fall 2024

[Acknowledgments Page](https://ds100.org/fa24/acks/)

## Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

In [1]:
pip install jupysql --upgrade

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.1 MB 365.1 kB/s eta 0:00:05
   ---------- ----------------------------- 0.5/2.1 MB 365.1 kB/s eta 0:00:05
   ---------- ----------------------------- 0.5/2.1 MB 365.1 kB/s eta 0:00:05
   --------------- ------------------------ 0.8/2.1 MB 372.9 kB/s eta 0:00:04
   --------------- ------------------------ 0.8/2.1 MB 372.9 kB/s e

### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [3]:
%load_ext sql

**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db` and DuckDB database `example_duck.db`.

In [4]:
%sql sqlite:///data/basic_examples.db --alias sqlite

Connecting to 'sqlite'

In [5]:
#%pip install duckdb-engine --quiet

In [6]:
%sql duckdb:///data/example_duck.db --alias duckdb

Connecting and switching to connection 'duckdb'

If you were connecting to an "enterprise data platform"

```python
from sqlalchemy import create_engine

snow_engine = create_engine(
    f"snowflake://{user}:{password}@{account_identifier}")
%sql snow_engine --alias snow

db_engine = create_engine(
  url = f"databricks://token:{access_token}@{server_hostname}?" +
        f"http_path={http_path}&catalog={catalog}&schema={schema}"
)
%sql db_engine --alias db
```

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [7]:
%%sql
SHOW TABLES;
SELECT * FROM Dragon;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

**Simple query, this time on two different lines.**

In [15]:
%%sql
SELECT *
FROM Dragon;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

### Approach #3: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [16]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("duckdb:///data/example_duck.db")

**2. Run a simple SQL query**

In [17]:
query = """
SELECT * 
FROM Dragon;
"""

df = pd.read_sql(query, engine)
df

name  year   cute
0    hiccup  2010   10.0
1    drogon  2011 -100.0
2  dragon 2  2019    0.0
3      puff  2010  100.0
4     smaug  2011    NaN

### Approach "3" -- Duck DB Special

Now that we are using DuckDB we can do something extra crazy:

In [1]:
import seaborn as sns
mpg = sns.load_dataset("mpg")
type(mpg)

pandas.core.frame.DataFrame

In [8]:
%%sql
SELECT * FROM mpg

Running query in 'duckdb'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.CatalogException) Catalog Error: Table with name mpg does not exist!
Did you mean "pg_am"?
LINE 1: SELECT * FROM mpg
                      ^
[SQL: SELECT * FROM mpg]
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community


That is right!! DuckDB can also see my dataframes in the python environment allowing me to do dataframe manipulation in SQL!


---

## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `example_duck.db` and explore the tables it stores.

In [36]:
%%sql
SELECT * FROM information_schema.tables

Running query in 'duckdb'

+---------------+--------------+------------+------------+------------------------------+----------------------+---------------------------+--------------------------+------------------------+--------------------+----------+---------------+---------------+
| table_catalog | table_schema | table_name | table_type | self_referencing_column_name | reference_generation | user_defined_type_catalog | user_defined_type_schema | user_defined_type_name | is_insertable_into | is_typed | commit_action | TABLE_COMMENT |
+---------------+--------------+------------+------------+------------------------------+----------------------+---------------------------+--------------------------+------------------------+--------------------+----------+---------------+---------------+
|  example_duck |     main     |    dish    | BASE TABLE |             None             |         None         |            None           |           None           |          None          |        YES         |    NO    |      None     |      None     |
|  example_duck |     main     |   dragon   | BASE TABLE |             None             |         None         |            None           |           None           |          None          |        YES         |    NO    |      None     |      None     |
|  example_duck |     main     |   scene    | BASE TABLE |             None             |         None         |            None           |           None           |          None          |        YES         |    NO    |      None     |      None     |
+---------------+--------------+------------+------------+------------------------------+----------------------+---------------------------+--------------------------+------------------------+--------------------+----------+---------------+---------------+

In [38]:
%%sql
SELECT * FROM information_schema.columns

Running query in 'duckdb'

+---------------+--------------+------------+-------------+------------------+----------------+-------------+-----------+--------------------------+------------------------+-------------------+-------------------------+---------------+--------------------+---------------+--------------------+-----------------------+----------------------+--------------------+-------------------+------------------+----------------+----------------+---------------+-------------+-------------+------------+----------+---------------+--------------+------------+---------------------+----------------+---------------------+-------------+---------------------+----------------+--------------------+------------------+------------------+----------------+--------------+-----------------------+--------------+----------------+
| table_catalog | table_schema | table_name | column_name | ordinal_position | column_default | is_nullable | data_type | character_maximum_length | character_octet_length | numeric_precision | numeric_precision_radix | numeric_scale | datetime_precision | interval_type | interval_precision | character_set_catalog | character_set_schema | character_set_name | collation_catalog | collation_schema | collation_name | domain_catalog | domain_schema | domain_name | udt_catalog | udt_schema | udt_name | scope_catalog | scope_schema | scope_name | maximum_cardinality | dtd_identifier | is_self_referencing | is_identity | identity_generation | identity_start | identity_increment | identity_maximum | identity_minimum | identity_cycle | is_generated | generation_expression | is_updatable | COLUMN_COMMENT |
+---------------+--------------+------------+-------------+------------------+----------------+-------------+-----------+--------------------------+------------------------+-------------------+-------------------------+---------------+--------------------+---------------+--------------------+-----------------------+----------------------+--------------------+-------------------+------------------+----------------+----------------+---------------+-------------+-------------+------------+----------+---------------+--------------+------------+---------------------+----------------+---------------------+-------------+---------------------+----------------+--------------------+------------------+------------------+----------------+--------------+-----------------------+--------------+----------------+
|  example_duck |     main     |    dish    |     name    |        1         |      None      |      NO     |  VARCHAR  |           None           |          None          |        None       |           None          |      None     |        None        |      None     |        None        |          None         |         None         |        None        |        None       |       None       |      None      |      None      |      None     |     None    |     None    |    None    |   None   |      None     |     None     |    None    |         None        |      None      |         None        |     None    |         None        |      None      |        None        |       None       |       None       |      None      |     None     |          None         |     None     |      None      |
|  example_duck |     main     |    dish    |     type    |        2         |      None      |     YES     |  VARCHAR  |           None           |          None          |        None       |           None          |      None     |        None        |      None     |        None        |          None         |         None         |        None        |        None       |       None       |      None      |      None      |      None     |     None    |     None    |    None    |   None   |      None     |     None     |    None    |         None        |      None      |         None        |     None    |         None        |      None      |        None        |       None       |       None       |      None      |     None     |          None   

### Getting Schema information with SQLAlchemy 
The way you list the tables varies across database platforms. For example, the statement:

```sql
SELECT * FROM information_schema.columns
```

It only works on Postgres-compatible databases.

For example, if we wanted to get the schema for tables in DuckDB, we would need the following:

In [40]:
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None
pd.read_sql("SELECT * FROM sqlite_schema", "duckdb:///data/example_duck.db")

type    name tbl_name  rootpage  \
0  table    dish     dish         0   
1  table  dragon   dragon         0   
2  table   scene    scene         0   

                                                                                                                                                                                  sql  
0                                                                                CREATE TABLE dish("name" VARCHAR PRIMARY KEY, "type" VARCHAR, "cost" INTEGER, CHECK(("cost" >= 0)));  
1                                                                             CREATE TABLE dragon("name" VARCHAR PRIMARY KEY, "year" INTEGER, cute INTEGER, CHECK(("year" >= 2000)));  
2  CREATE TABLE scene(id INTEGER PRIMARY KEY, biome VARCHAR NOT NULL, city VARCHAR NOT NULL, visitors INTEGER, created_at TIMESTAMP DEFAULT(current_date()), CHECK((visitors >= 0)));

Fortunately, SQLAlchemy has some generic tools that will be helpful regardless of what database platform you use.

In [41]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

['dish', 'dragon', 'scene']

In [42]:
inspector.get_columns('scene')

[{'name': 'id',
  'type': Integer(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'biome',
  'type': String(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'city',
  'type': String(),
  'nullable': False,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'visitors',
  'type': Integer(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None},
 {'name': 'created_at',
  'type': TIMESTAMP(),
  'nullable': True,
  'default': 'current_date()',
  'autoincrement': False,
  'comment': None}]

Same with SQLite.

In [44]:
sqlite_engine = sqlalchemy.create_engine("sqlite:///data/basic_examples.db")
inspect(sqlite_engine).get_columns("scene")

[{'name': 'id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 1},
 {'name': 'biome',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'city',
  'type': TEXT(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'visitors',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'created_at',
  'type': DATETIME(),
  'nullable': True,
  'default': "DATETIME('now')",
  'primary_key': 0}]

### Example of table creation with interesting constraints

More advanced example of creating tables with primary and foreign key constraints:

In [45]:
%%sql

DROP TABLE IF EXISTS grade;
DROP TABLE IF EXISTS assignment;
DROP TABLE IF EXISTS student;


CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name VARCHAR,
    email VARCHAR
);

CREATE TABLE assignment (
    assignment_id INTEGER PRIMARY KEY,
    description VARCHAR
);

CREATE TABLE grade (
    student_id INTEGER,
    assignment_id INTEGER,
    score REAL CHECK (score > 0 AND score <= 100),
    FOREIGN KEY (student_id) REFERENCES student(student_id),
    FOREIGN KEY (assignment_id) REFERENCES assignment(assignment_id)
);

INSERT INTO student VALUES
(123, 'JoeyG', 'jegonzal@berkeley.edu'),
(456, 'NargesN', 'norouzi@berkeley.edu');

INSERT INTO assignment VALUES
(1, 'easy assignment'),
(2, 'hard assignment');

Running query in 'duckdb'

+-------+
| Count |
+-------+
+-------+

In [48]:
%%sql 
INSERT INTO grade VALUES
(123, 1, 80),
(123, 2, 42),
(456, 2, 100);

Running query in 'duckdb'

+-------+
| Count |
+-------+
+-------+

In [47]:
%sql SELECT * FROM grade;

Running query in 'duckdb'

+------------+---------------+-------+
| student_id | assignment_id | score |
+------------+---------------+-------+
|    123     |       1       |  80.0 |
|    123     |       2       |  42.0 |
|    456     |       2       | 100.0 |
+------------+---------------+-------+

<br/>

---

## Basic Queries

### SELECT and FROM

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: Specifies the column(s) to return in the output.
* `FROM`: Specifies the database table from which to extract data.

In [49]:
%%sql
SELECT * FROM Dragon;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

In [50]:
%%sql
SELECT cute, year FROM Dragon;

Running query in 'duckdb'

+------+------+
| cute | year |
+------+------+
|  10  | 2010 |
| -100 | 2011 |
|  0   | 2019 |
| 100  | 2010 |
| None | 2011 |
+------+------+

### Aliasing with `AS`

In [51]:
%%sql
SELECT cute AS cuteness,
       year AS "birth year"
FROM Dragon;

Running query in 'duckdb'

+----------+------------+
| cuteness | birth year |
+----------+------------+
|    10    |    2010    |
|   -100   |    2011    |
|    0     |    2019    |
|   100    |    2010    |
|   None   |    2011    |
+----------+------------+

### Uniqueness with `DISTINCT`

In [52]:
%%sql
SELECT DISTINCT year
FROM Dragon;

Running query in 'duckdb'

+------+
| year |
+------+
| 2010 |
| 2019 |
| 2011 |
+------+

### Filtering with `WHERE`

In [54]:
%%sql
SELECT name, year
FROM Dragon
WHERE cute <=0;

Running query in 'duckdb'

+----------+------+
|   name   | year |
+----------+------+
|  drogon  | 2011 |
| dragon 2 | 2019 |
+----------+------+

In [55]:
%%sql
SELECT name, cute, year
FROM Dragon
WHERE cute > 0 OR year > 2013;

Running query in 'duckdb'

+----------+------+------+
|   name   | cute | year |
+----------+------+------+
|  hiccup  |  10  | 2010 |
|   puff   | 100  | 2010 |
| dragon 2 |  0   | 2019 |
+----------+------+------+

In [56]:
%%sql
SELECT name, year
FROM Dragon 
WHERE name IN ('puff', 'hiccup');

Running query in 'duckdb'

+--------+------+
|  name  | year |
+--------+------+
|  puff  | 2010 |
| hiccup | 2010 |
+--------+------+

#### Checking against NULL

In [59]:
%%sql
SELECT name, cute
FROM Dragon
WHERE cute IS NOT NULL;

Running query in 'duckdb'

+----------+------+
|   name   | cute |
+----------+------+
|  hiccup  |  10  |
|  drogon  | -100 |
| dragon 2 |  0   |
|   puff   | 100  |
+----------+------+

### Ordering data using `ORDER BY`

In [60]:
%%sql
SELECT *
FROM Dragon
ORDER BY cute DESC;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|   puff   | 2010 | 100  |
|  hiccup  | 2010 |  10  |
| dragon 2 | 2019 |  0   |
|  drogon  | 2011 | -100 |
|  smaug   | 2011 | None |
+----------+------+------+

### Restricting output with `LIMIT` and `OFFSET`

In [63]:
%%sql
SELECT *
FROM Dragon;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

In [64]:
%%sql
SELECT *
FROM Dragon
LIMIT 2;

Running query in 'duckdb'

+--------+------+------+
|  name  | year | cute |
+--------+------+------+
| hiccup | 2010 |  10  |
| drogon | 2011 | -100 |
+--------+------+------+

In [65]:
%%sql
SELECT *
FROM Dragon
LIMIT 2
OFFSET 2;

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
+----------+------+------+

## Sampling

What if we wanted a random sample:

In [71]:
%%sql
SELECT *
FROM Dragon
ORDER BY RANDOM() 
LIMIT 2

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
+----------+------+------+

In [86]:
#reservoir: 指定使用 reservoir 抽样算法。Reservoir 抽样是一种随机抽样算法，适用于大数据集。
#(2 ROWS): 指定要抽取的样本行数为 2。
#REPEATABLE: 用于确保每次执行查询时，抽样的结果都是相同的。
#(42): 指定一个种子值（seed）

In [87]:
%%sql
SELECT * 
FROM Dragon USING SAMPLE reservoir(2 ROWS) REPEATABLE (42) ;


Running query in 'duckdb'

+--------+------+------+
|  name  | year | cute |
+--------+------+------+
| hiccup | 2010 |  10  |
| smaug  | 2011 | None |
+--------+------+------+

## Grouping Data with `GROUP BY`

In [88]:
%%sql
SELECT *
FROM Dish;

Running query in 'duckdb'

+------------+-----------+------+
|    name    |    type   | cost |
+------------+-----------+------+
|  ravioli   |   entree  |  10  |
|   ramen    |   entree  |  13  |
|    taco    |   entree  |  7   |
|  edamame   | appetizer |  4   |
|   fries    | appetizer |  4   |
| potsticker | appetizer |  4   |
| ice cream  |  dessert  |  5   |
+------------+-----------+------+

A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

In [89]:
%%sql
SELECT type
FROM Dish;

Running query in 'duckdb'

+-----------+
|    type   |
+-----------+
|   entree  |
|   entree  |
|   entree  |
| appetizer |
| appetizer |
| appetizer |
|  dessert  |
+-----------+

In [90]:
%%sql
SELECT type
FROM Dish
GROUP BY type;

Running query in 'duckdb'

+-----------+
|    type   |
+-----------+
| appetizer |
|   entree  |
|  dessert  |
+-----------+

In [35]:
%%sql
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

Running query in 'duckdb'

+-----------+-------------+
|    type   | sum("cost") |
+-----------+-------------+
|   entree  |      30     |
|  dessert  |      5      |
| appetizer |      12     |
+-----------+-------------+

In [93]:
%%sql
SELECT type, 
       SUM(cost), 
       MIN(cost),
       MIN(name)
FROM Dish
GROUP BY type;

Running query in 'duckdb'

+-----------+-------------+-------------+-------------+
|    type   | sum("cost") | min("cost") | min("name") |
+-----------+-------------+-------------+-------------+
|   entree  |      30     |      7      |    ramen    |
|  dessert  |      5      |      5      |  ice cream  |
| appetizer |      12     |      4      |   edamame   |
+-----------+-------------+-------------+-------------+

In [94]:
%%sql
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

Running query in 'duckdb'

+------+-------------+
| year | count(cute) |
+------+-------------+
| 2010 |      2      |
| 2011 |      1      |
| 2019 |      1      |
+------+-------------+

In [95]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

Running query in 'duckdb'

+------+--------------+
| year | count_star() |
+------+--------------+
| 2010 |      2       |
| 2011 |      2       |
| 2019 |      1       |
+------+--------------+

## Working with the `sql` results as Python variables

By default, executing a query with a magic command produces output but doesn't save it into any Python variable:

In [96]:
%sql SELECT * FROM Dragon

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

This can be inconvenient if you later want to do further processing of these data in Python.

### Storing one-line `%sql` queries

For simple one-line queries, you can use IPython's ability to store the result of a magic command like `%sql` as if it were any other Python statement and save the output to a variable:

In [97]:
dragon_table = %sql SELECT * FROM Dragon
dragon_table

Running query in 'duckdb'

+----------+------+------+
|   name   | year | cute |
+----------+------+------+
|  hiccup  | 2010 |  10  |
|  drogon  | 2011 | -100 |
| dragon 2 | 2019 |  0   |
|   puff   | 2010 | 100  |
|  smaug   | 2011 | None |
+----------+------+------+

As noted above, the result of the query is a Python variable of type `ResultSet`, more specifically:

In [98]:
type(dragon_table)

sql.run.resultset.ResultSet

You need to manually convert it to a `pandas` DataFrame if you want to do pandas-things with its content:

In [99]:
dragon_df = dragon_table.DataFrame()
dragon_df

name  year   cute
0    hiccup  2010   10.0
1    drogon  2011 -100.0
2  dragon 2  2019    0.0
3      puff  2010  100.0
4     smaug  2011    NaN

You can configure `jupysql` to _automatically_ convert all outputs to Pandas DataFrames. This can be handy if you intend all your Python-side work to be done with Pandas, as it saves you from manually having to call `.DataFrame()` first on all outputs. On the other hand, you don't get access to the original SQL `ResultSet` object, which has a number of interesting properties and capabilities. You can learn more about those in the [jupysql documentation](https://jupysql.ploomber.io).

For now, let's turn this on so you can see how this simplified "pandas all the way" workflow looks like:

In [13]:
%config SqlMagic.autopandas = True

In [14]:
dragon_df = %sql SELECT * FROM Dragon
dragon_df

Running query in 'duckdb'

name  year   cute
0    hiccup  2010   10.0
1    drogon  2011 -100.0
2  dragon 2  2019    0.0
3      puff  2010  100.0
4     smaug  2011    NaN

In [102]:
type(dragon_df)

pandas.core.frame.DataFrame

### Storing multi-line `%%sql` queries

For a more complex query that won't fit in one line, such as for example:

In [46]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

Running query in 'duckdb'

year  count_star()
0  2010             2
1  2011             2
2  2019             1

You can use the `variable <<` syntax in `jupysql` to store its output (this will honor your `autopandas` state and store either a `sql.run.ResultState` or a Pandas `DataFrame`):

In [16]:
%%sql dragon_years <<
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

Running query in 'duckdb'

In [17]:
type(dragon_years)

pandas.core.frame.DataFrame